In [1]:
from dotenv import load_dotenv
import os
print("OPENAI_API_KEY: ", os.getenv("OPENAI_API_KEY")[:10])

OPENAI_API_KEY:  sk-proj-pw


# PromptTemplate

In [2]:
from langchain_core.prompts import PromptTemplate

In [3]:
template = "What is a good name for a company that makes {product}?"
prompt = PromptTemplate.from_template(template)

In [4]:
formatted_prompt = prompt.format(product="colorful socks")
print(formatted_prompt)

What is a good name for a company that makes colorful socks?


# ChatOpenAI

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
llm = ChatOpenAI(model="gpt-4.1-nano", max_tokens=50, temperature=0.2)

In [ ]:
response = llm.invoke([("human", "Hello, who are you?")])
print(response)
print(response.content)    # 메세지만 보기.

content="Hello! I'm an AI language model here to assist you. How can I help today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-Bk1y67BCopLiT6AJHy4R62CGhROos', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--c2ba6176-7d2b-4142-93b2-55761bb5ae9c-0' usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Hello! I'm an AI language model here to assist you. How can I help today?


# Runnable
- Runnable 인터페이스: 실행 가능 구성 요소
- 서로 다른 컴포넌트들이 호환가능한 형태로 결합할 수 있게 만들어 줌.
- 클래스의 형타입을 통일시키는 역할 -> 호환, 결합 가능

# StrOutputParser

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
parser = StrOutputParser()
parsed_response = parser.invoke(response)
print(parsed_response)

Hello! I'm an AI language model here to assist you. How can I help today?


# Pipline, LCEL(LangChain Expression Language)

In [ ]:
chain = prompt | llm | parser   # PromptTemplate -> ChatOpenAI -> StrOutputParser 실행 순서로 출력

In [ ]:
result = chain.invoke({"product": "decaffeinated coffee"})
print(result)

Sure! Here are some creative and appealing name ideas for a company that makes decaffeinated coffee:

1. Calm Brew
2. Serenity Coffee Co.
3. Zero Buzz Coffee
4. Tranquil Beans
5. Purely Decaf


# LangChain으로 토큰 사용량 추적

In [16]:
print("토큰 사용 정보: ", response.usage_metadata)

토큰 사용 정보:  {'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [17]:
!uv add langchain_community

Resolved 58 packages in 665ms
 Downloaded langchain-community
Prepared 14 packages in 5.63s
Installed 15 packages in 432ms
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.13
 + aiosignal==1.3.2
 + dataclasses-json==0.6.7
 + frozenlist==1.7.0
 + httpx-sse==0.4.0
 + langchain-community==0.3.25
 + marshmallow==3.26.1
 + multidict==6.5.0
 + mypy-extensions==1.1.0
 + numpy==2.3.0
 + propcache==0.3.2
 + pydantic-settings==2.9.1
 + typing-inspect==0.9.0
 + yarl==1.20.1


In [18]:
# 콜백을 통한 누적 토큰 추적
from langchain_community.callbacks import get_openai_callback

In [19]:
with get_openai_callback() as cb:
  res1 = llm.invoke("조치원의 오늘 날씨는 어때?")
  print("응답: ", res1.content[:30], "...")
  res2 = llm.invoke("파이썬으로 리스트 만드는 방법을 예시로 알려줘")
  print("응답: ", res2.content[:30], "...")

응답:  죄송하지만, 현재 실시간 날씨 정보를 제공할 수 없습니 ...
응답:  물론입니다! 파이썬에서 리스트를 만드는 방법에는 여러  ...


In [21]:
print(f"총 토큰 수: {cb.total_tokens}")
print(f"프롬프트 토큰:  {cb.prompt_tokens}")
print(f"응답 토큰: {cb.completion_tokens}")
print(f"총 비용: {cb.total_cost:.6f}")

총 토큰 수: 129
프롬프트 토큰:  38
응답 토큰: 91
총 비용: 0.000040


# Langchain LLM 응답 캐싱

In [22]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemoryCache 설정
set_llm_cache(InMemoryCache())

In [23]:
query = "재미있는 과학유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1: ", result1.content)

응답1:  물리학자가 친구에게 농담을 던졌어요:

"왜 전자는 항상 파티에 늦게 와?"  
"왜?"  
"왜냐하면, 그는 항상 '음전하' 때문에 '전기'를


In [24]:
result2 = llm.invoke(query)
print("응답2: ", result2.content)

응답2:  물리학자가 친구에게 농담을 던졌어요:

"왜 전자는 항상 파티에 늦게 와?"  
"왜?"  
"왜냐하면, 그는 항상 '음전하' 때문에 '전기'를


In [25]:
# SQLite를 이용 디스크 기반 캐시 설정
import os
from langchain_community.cache import SQLiteCache

In [26]:
# 기존 캐시 DB 삭제(실습을 위해서!!!)
if os.path.exists(".langchain_cache.db"):
  os.remove(".langchain_cache.db")
  
# SQLiteCache 설정
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

In [27]:
query = "재미있는 과학유머 하나 알려줘."
result1 = llm.invoke(query)
print("응답1: ", result1.content)

응답1:  물리학자가 친구에게 농담을 던졌어요:

"왜 전자는 항상 파티에 늦게 와?"  
"왜?"  
"왜냐하면, 그는 항상 '음전하' 때문에 '음'이 붙


In [28]:
result2 = llm.invoke(query)
print("응답2: ", result2.content)

응답2:  물리학자가 친구에게 농담을 던졌어요:

"왜 전자는 항상 파티에 늦게 와?"  
"왜?"  
"왜냐하면, 그는 항상 '음전하' 때문에 '음'이 붙
